In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

In [2]:
import hopsworks

# Login to the Hopsworks project using your API key
project = hopsworks.login(
    api_key_value="mAaHAWHefj9ff3oE.OgcnSgFgldipqzmX8pbMMnkM2sEY2SpIeGVrrcuIlNwwM5I89Oe3jZj2C8CE8oTm")

# Get the feature store instance
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1047702
2024-10-14 16:53:55,996 WARNING: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead

2024-10-14 16:53:55,997 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
feature_group_final = fs.get_feature_group(name='final_features', version=1)

In [4]:
df = feature_group_final.read()
df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.77s) 


,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_precip,...,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight,unique_id,event_date
0,14560694,R-fe016941,2019-01-13 07:00:00+00:00,2019-01-14 01:39:00+00:00,0,59.600000,10.800000,71.000000,1014.400000,0.0,...,female,47.0,7.0,proactive,8.0,14560694.0,56.770000,1,1869,2024-10-06 00:00:00+00:00
1,21162393,R-d04e90bc,2019-01-04 07:00:00+00:00,2019-01-04 21:32:24+00:00,1,72.000000,5.500000,65.750000,1018.750000,0.0,...,male,45.0,1.0,conservative,8.0,21162393.0,41.369999,0,3626,2024-10-06 00:00:00+00:00
2,79515526,R-564643b5,2019-01-01 07:00:00+00:00,2019-01-01 07:25:48+00:00,0,81.500000,11.000000,82.500000,1009.000000,0.0,...,female,52.0,20.0,conservative,7.0,79515526.0,50.380001,0,6239,2024-10-06 00:00:00+00:00
3,25976643,R-4e6c7c29,2019-01-28 07:00:00+00:00,2019-01-29 06:29:24+00:00,0,65.333333,4.333333,78.666667,1014.000000,0.0,...,male,43.0,0.0,proactive,8.0,25976643.0,56.790001,1,7606,2024-10-06 00:00:00+00:00
4,44799976,R-74abbc2c,2019-01-16 07:00:00+00:00,2019-01-17 16:39:00+00:00,1,51.000000,8.857143,56.857143,1019.000000,0.0,...,male,50.0,15.0,proactive,5.0,44799976.0,64.820000,1,7349,2024-10-06 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10477,59856374,R-745d907e,2019-01-25 07:00:00+00:00,2019-01-25 14:29:24+00:00,0,54.000000,9.000000,78.000000,1013.000000,0.0,...,male,47.0,12.0,proactive,8.0,59856374.0,57.040001,0,3680,2024-10-06 00:00:00+00:00
10478,33573400,R-695dcbf7,2019-02-10 07:00:00+00:00,2019-02-12 05:43:48+00:00,1,48.888889,8.222222,80.111111,1015.888889,0.0,...,male,45.0,7.0,proactive,8.0,33573400.0,58.619999,1,5135,2024-10-06 00:00:00+00:00
10479,31120899,R-a0fcbe48,2019-01-25 07:00:00+00:00,2019-01-25 12:37:12+00:00,0,47.666667,5.666667,83.333333,1011.666667,0.0,...,male,48.0,12.0,conservative,6.0,31120899.0,52.810001,0,5923,2024-10-06 00:00:00+00:00
10480,14849907,R-1d7b758f,2019-01-04 07:00:00+00:00,2019-01-04 08:03:36+00:00,0,68.000000,3.500000,74.000000,1010.000000,0.0,...,female,46.0,16.0,conservative,8.0,14849907.0,48.099998,0,10453,2024-10-06 00:00:00+00:00


In [5]:
df = df.drop(columns=['unique_id', 'event_date'])

In [6]:
df['departure_date'] = pd.to_datetime(
    df['departure_date']).dt.strftime('%Y-%m-%d %H:%M:%S')
df['estimated_arrival'] = pd.to_datetime(
    df['estimated_arrival']).dt.strftime('%Y-%m-%d %H:%M:%S')
df['estimated_arrival_nearest_hour'] = pd.to_datetime(
    df['estimated_arrival_nearest_hour']).dt.strftime('%Y-%m-%d %H:%M:%S')
df['departure_date_nearest_hour'] = pd.to_datetime(
    df['departure_date_nearest_hour']).dt.strftime('%Y-%m-%d %H:%M:%S')

In [7]:
cts_cols = ['route_avg_temp', 'route_avg_wind_speed', 'route_precip',
            'route_avg_humidity', 'route_visibility', 'route_avg_pressure',
            'distance', 'average_hours', 'origin_temp', 'origin_wind_speed',
            'origin_precip', 'origin_humidity', 'origin_visibility', 'origin_pressure',
            'dest_temp', 'dest_wind_speed', 'dest_precip',
            'dest_humidity', 'dest_visibility', 'dest_pressure',
            'avg_no_of_vehicles', 'truck_age', 'load_capacity_pounds', 'mileage_mpg',
            'age', 'experience', 'average_speed_mph']

cat_cols = ['route_description', 'origin_description', 'dest_description',
            'accident', 'fuel_type', 'gender', 'driving_style', 'ratings', 'is_midnight']

target = ['delay']

In [8]:
df['estimated_arrival'] = pd.to_datetime(
    df['estimated_arrival'], errors='coerce')

In [9]:
train_df = df[df['estimated_arrival'] <= pd.to_datetime('2019-01-30')]
train_df.shape

(7299, 48)

In [10]:
validation_df = df[(df['estimated_arrival'] > pd.to_datetime('2019-01-30')) &

                   (df['estimated_arrival'] <= pd.to_datetime('2019-02-07'))]
validation_df.shape

(1910, 48)

In [11]:
test_df = df[df['estimated_arrival'] > pd.to_datetime('2019-02-07')]
test_df.shape

(1273, 48)

In [12]:
X_train = train_df[cts_cols+cat_cols]

y_train = train_df['delay']

In [13]:
X_valid = validation_df[cts_cols + cat_cols]

y_valid = validation_df['delay']

In [14]:
X_test = test_df[cts_cols+cat_cols]

y_test = test_df['delay']

In [15]:
encoder_columns = ['route_description', 'origin_description',
                   'dest_description', 'fuel_type', 'gender', 'driving_style']

In [16]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [17]:
# Check the unique values in the training set for each column before fitting the encoder
for col in encoder_columns:
    print(f"Unique values in {col}: {X_train[col].nunique()}")

# Fit the OneHotEncoder to the training data
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder.fit(X_train[encoder_columns])

# Get the names of the encoded features
encoded_features = encoder.get_feature_names_out(encoder_columns)
print(f"Encoded feature names: {encoded_features}")

# Transform the datasets
X_train_encoded = pd.DataFrame(encoder.transform(
    X_train[encoder_columns]), columns=encoded_features, index=X_train.index)
X_valid_encoded = pd.DataFrame(encoder.transform(
    X_valid[encoder_columns]), columns=encoded_features, index=X_valid.index)
X_test_encoded = pd.DataFrame(encoder.transform(
    X_test[encoder_columns]), columns=encoded_features, index=X_test.index)

# Debugging to check the shape of the transformed data
print(f"X_train_encoded shape: {X_train_encoded.shape}")
print(f"X_valid_encoded shape: {X_valid_encoded.shape}")
print(f"X_test_encoded shape: {X_test_encoded.shape}")

# Concatenate the encoded features back to the original DataFrames
X_train = pd.concat(
    [X_train.drop(encoder_columns, axis=1), X_train_encoded], axis=1)
X_valid = pd.concat(
    [X_valid.drop(encoder_columns, axis=1), X_valid_encoded], axis=1)
X_test = pd.concat(
    [X_test.drop(encoder_columns, axis=1), X_test_encoded], axis=1)

Unique values in route_description: 43
Unique values in origin_description: 27
Unique values in dest_description: 38
Unique values in fuel_type: 2
Unique values in gender: 2
Unique values in driving_style: 2
Encoded feature names: ['route_description_Blizzard' 'route_description_Blowing snow'
 'route_description_Clear' 'route_description_Cloudy'
 'route_description_Fog' 'route_description_Freezing drizzle'
 'route_description_Freezing fog' 'route_description_Heavy rain'
 'route_description_Heavy rain at times' 'route_description_Heavy snow'
 'route_description_Light drizzle' 'route_description_Light freezing rain'
 'route_description_Light rain' 'route_description_Light rain shower'
 'route_description_Light sleet' 'route_description_Light sleet showers'
 'route_description_Light snow' 'route_description_Mist'
 'route_description_Moderate or heavy freezing rain'
 'route_description_Moderate or heavy rain shower'
 'route_description_Moderate or heavy rain with thunder'
 'route_descripti

In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit the scaler on training data for the continuous columns
scaler.fit(X_train[cts_cols])

# Transform the continuous columns in all datasets
X_train_scaled = pd.DataFrame(scaler.transform(
    X_train[cts_cols]), columns=cts_cols, index=X_train.index)
X_valid_scaled = pd.DataFrame(scaler.transform(
    X_valid[cts_cols]), columns=cts_cols, index=X_valid.index)
X_test_scaled = pd.DataFrame(scaler.transform(
    X_test[cts_cols]), columns=cts_cols, index=X_test.index)

# Concatenate the scaled continuous features back to the original DataFrames
X_train = pd.concat([X_train.drop(cts_cols, axis=1), X_train_scaled], axis=1)
X_valid = pd.concat([X_valid.drop(cts_cols, axis=1), X_valid_scaled], axis=1)
X_test = pd.concat([X_test.drop(cts_cols, axis=1), X_test_scaled], axis=1)

# Debugging to check the shape of the final data
print(f"Final X_train shape: {X_train.shape}")
print(f"Final X_valid shape: {X_valid.shape}")
print(f"Final X_test shape: {X_test.shape}")

Final X_train shape: (7299, 144)
Final X_valid shape: (1910, 144)
Final X_test shape: (1273, 144)


In [19]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb

In [28]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Initialize the XGBoost classifier
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Define hyperparameters for Grid Search
param_grid = {
    'n_estimators': [100, 200],  # Number of trees
    'max_depth': [3, 5, 7],  # Depth of trees
    'learning_rate': [0.01, 0.1, 0.2],  # Learning rate
    'subsample': [0.8, 1.0],  # Fraction of samples used per tree
    'colsample_bytree': [0.8, 1.0],  # Fraction of features used per tree
    'reg_lambda': [1, 10, 100],  # L2 regularization strength
    'reg_alpha': [0, 0.1, 1]  # L1 regularization strength
}

# Create a Grid Search CV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           scoring='accuracy', cv=5, n_jobs=-1)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Predict on training set
y_train_pred = best_model.predict(X_train)

# Predict on validation set
y_valid_pred = best_model.predict(X_valid)

# Predict on test set
y_test_pred = best_model.predict(X_test)

# Calculate accuracy and classification reports
train_accuracy = accuracy_score(y_train, y_train_pred)
valid_accuracy = accuracy_score(y_valid, y_valid_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

train_report = classification_report(y_train, y_train_pred)
valid_report = classification_report(y_valid, y_valid_pred)
test_report = classification_report(y_test, y_test_pred)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Training accuracy: {train_accuracy}")
print(f"Validation accuracy: {valid_accuracy}")
print(f"Test accuracy: {test_accuracy}")

print(f"\nClassification report for training set:\n{train_report}")
print(f"Classification report for validation set:\n{valid_report}")
print(f"Classification report for test set:\n{test_report}")


2024-10-14 20:25:29,460 WARNING: DeprecationWarning: This process (pid=16505) is multi-threaded, use of fork() may lead to deadlocks in the child.



/Users/rammaruthi/Desktop/Truck/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:25:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/rammaruthi/Desktop/Truck/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:25:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/rammaruthi/Desktop/Truck/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:25:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/rammaruthi/Desktop/Truck/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:25:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" 

2024-10-14 20:50:35,566 WARNING: UserWarning: [20:50:35] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200, 'reg_alpha': 1, 'reg_lambda': 1, 'subsample': 1.0}
Training accuracy: 0.9154285714285715
Validation accuracy: 0.49466666666666664
Test accuracy: 0.5066666666666667

Classification report for training set:
              precision    recall  f1-score   support

           0       0.92      0.91      0.92      3517
           1       0.91      0.92      0.92      3483

    accuracy                           0.92      7000
   macro avg       0.92      0.92      0.92      7000
weighted avg       0.92      0.92      0.92      7000

Classification report for validation set:
              precision    recall  f1-score   support

           0       0.46      0.47      0.47       707
           1       0.52      0.52      0.